In [1]:
import itertools
import math as m
import os 
import networkx as nx

In [2]:
def read_input(graphfile):
    
    trip_data = open(graphfile,'r').read().split('\n')
    i = 0
    listOfGraphs = {}
    k = 0
    
    while(True):
        if "#" in trip_data[i]:
            i = i+1
            N = int(trip_data[i])
            edges = list()
            while(True):
                i = i+1;
                if "#" in trip_data[i]:
                    break;
                if "" == trip_data[i]:
                    break;
                line = trip_data[i].split(" ")
                edges.append((line[0],line[1],line[2],line[3]))
                if i >= len(trip_data)-1:
                    break;
            G = {'Nodes':N,'list of edges': edges}
            listOfGraphs[k] = G
            k +=1 
        if i >= len(trip_data)-1:
                    break; 

    return listOfGraphs;
    

In [3]:
def flowMultipleDecomposition(data,K):
    
    
    #libraries
    import gurobipy as gp
    from gurobipy import GRB
    
    # calculate the minimal flow decomposition based on such graph
    V = data['vertices']
    E = data['edges']
    f_low = data['flows_low']
    f_up = data['flows_up']
    W = 1e5
    S = {str(min(map(int,V)))}
    D = {str(max(map(int,V)))}
    
    
    try:
        #create extra sets
        T = [(i,j,k) for (i,j) in E for k in range(0,K)]
        SC = [k for k in range(0,K)]
        
        # Create a new model
        model = gp.Model("MFD")
        model.Params.LogToConsole = 0

        # Create variables
        x = model.addVars(T,vtype=GRB.BINARY, name="x")
        w = model.addVars(SC,vtype=GRB.INTEGER, name="w",lb=0)
        z = model.addVars(T,vtype=GRB.CONTINUOUS, name="z",lb=0)
    
        model.setObjective(GRB.MINIMIZE)
       
        # flow conservation
        model.addConstrs(gp.quicksum(x[i,j,k] for j in V if i!=j and(i,j) in E) - gp.quicksum(x[j,i,k] for j in V if i!=j and (j,i) in E) == 1 for k in range(0,K) for i in V if i in S)  
        model.addConstrs(gp.quicksum(x[i,j,k] for j in V if i!=j and (i,j) in E) - gp.quicksum(x[j,i,k] for j in V if i!=j and (j,i) in E) == -1 for k in range(0,K) for i in V if i in D)
        model.addConstrs(gp.quicksum(x[i,j,k] for j in V if i!=j and (i,j) in E) - gp.quicksum(x[j,i,k] for j in V if i!=j and (j,i) in E) == 0 for k in range(0,K) for i in V if i not in D and i not in S)

        # flow balance inexact
        model.addConstrs(f_low[i,j] <= gp.quicksum(z[i,j,k] for k in range(0,K)) for (i,j) in E)
        model.addConstrs(f_up[i,j] >= gp.quicksum(z[i,j,k] for k in range(0,K)) for (i,j) in E)

        # linearization
        model.addConstrs(z[i,j,k] <= W*x[i,j,k] for(i,j) in E for k in range(0,K))
        model.addConstrs(w[k] - (1 - x[i,j,k])*W <= z[i,j,k] for(i,j) in E for k in range(0,K))
        model.addConstrs(z[i,j,k] <= w[k] for(i,j) in E for k in range(0,K))
        model.addConstrs(z[i,j,k] >= 0 for(i,j) in E for k in range(0,K))
        
        # objective function
        model.write("MFD.lp")
        model.optimize()
        
        if model.status == GRB.OPTIMAL:
            data['message'] = 'solved'
            data['runtime'] = model.Runtime;
            data['weights'] = K
        if model.status == GRB.INFEASIBLE:
            data['message'] = 'unsolved'
        
    except gp.GurobiError as e:
        print('Error code ' + str(e.errno) + ': ' + str(e))

    except AttributeError:
        print('Encountered an attribute error')
    
    return data;
    

In [4]:
def FD_Algorithm(data):
    
    listOfEdges = data['edges']
    solutionMap = data['graph']
    solutionSet = 0
    Kmin = data['minK']
    solutionWeights = 0

    for i in range(1,len(listOfEdges)+1):
        data = flowMultipleDecomposition(data,i)
        if data['message'] == "solved":
            break;
        
    
    return data

In [5]:
def SolveInstances(Graphs,outfile):
    
    fp = open(outfile,'w+')
    
    for i in range(0,1): 
        
        f_low = {}
        f_up = {}
        Edges = set()
        V = set()
        listOfEdges = Graphs[i]['list of edges']
        for k in range(0,len(listOfEdges)):
            (a,b,c,d) = (listOfEdges[k])
            Edges.add((a,b))
            V.add(a)
            V.add(b)
            f_low[a,b] = int(float(c))
            f_up[a,b] = int(float(d))
            
        
        # creation of graphs
        G = nx.DiGraph()
        G.add_edges_from(Edges,weights = f_low)
        G.add_nodes_from(V)
        
        
        # calculating topological order
        G_top = nx.topological_sort(G)
        G_top_list = list(nx.topological_sort(G))
        topological_cuts = sorted(G.degree, key=lambda x: x[1], reverse=True)[0][1]
        # definition of data
        
        data = {'edges' : Edges,
                'flows_low' : f_low,
                'flows_up': f_up,
                'vertices' : V,
                'graph' : G,
                'Kmax' : len(Edges),
                'weights' : {},
                'message': {},
                'minK': topological_cuts,
                'runtime': 0,
        }
        
        data = FD_Algorithm(data)        
        
        print(data['weights'],data['runtime'],file=fp)
        
    
    return 0

In [6]:
path = "./Gurobi/Inexact/"
text_files = [f for f in os.listdir(path) if f.endswith('.graph')]
outputfile = "results_gurobi_inexact.txt"

for i in range(0,len(text_files)):    
    filename = text_files[i]
    data = read_input(''.join([path,filename]))
    data = SolveInstances(data,outputfile)
    
print("Done")

Academic license - for non-commercial use only - expires 2022-10-10
Using license file /Users/cunhfern/gurobi.lic
Done
